In [3]:
!pip install xgboost


   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/72.0 MB 1.2 MB/s eta 0:01:00
    --------------------------------------- 1.0/72.0 MB 1.7 MB/s eta 0:00:43
    --------------------------------------- 1.0/72.0 MB 1.7 MB/s eta 0:00:43
    --------------------------------------- 1.0/72.0 MB 1.7 MB/s eta 0:00:43
    --------------------------------------- 1.6/72.0 MB 1.2 MB/s eta 0:00:58
   - -------------------------------------- 2.1/72.0 MB 1.5 MB/s eta 0:00:48
   - -------------------------------------- 2.6/72.0 MB 1.6 MB/s eta 0:00:45
   - -------------------------------------- 2.6/72.0 MB 1.6 MB/s eta 0:00:45
   - -------------------------------------- 2.6/72.0 MB 1.6 MB/s eta 0:00:45
   - -------------------------------------- 2.9/72.0 MB 1.3 MB/s eta 0:00:53
   - ---------------

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,confusion_matrix

In [5]:
df = pd.read_csv("crime through social media.csv")

In [6]:
X = df.drop(columns=["Click_Leads_To_Malicious_Site", "User_ID"])
y = df["Click_Leads_To_Malicious_Site"]

In [7]:
X

,Age_Group,Gender,Location,Device_Type,Social_Media_Platform,Two_Factor_Authentication,Saved_Password,Account_Privacy,Frequent_New_Device_Logging,Connection_Type,...,Payment_Mode,Cross_Channel_Usage,Season,Suspicious_Activity,Clicked_Ad,Ad_Platform,Ad_Target_Domain,Domain_Reputation,Domain_Length,Domain_Entropy
0,35-44,Female,Jharkhand,Mobile,Twitter,Yes,No,Private,No,Open WiFi,...,Digital Wallet,Yes,Holi,1,Yes,Instagram,banking.example,Good,15,3.507
1,18-24,Female,Uttar Pradesh,Mobile,Facebook,Yes,Yes,Private,No,Private,...,Digital Wallet,No,Diwali,1,No,NaN,NaN,NaN,0,0.000
2,35-44,Other,Gujarat,Mobile,Instagram,Yes,Yes,Public,No,Private,...,Credit Card,No,navaratri,0,No,NaN,NaN,NaN,0,0.000
3,18-24,Female,Uttar Pradesh,Tablet,WhatsApp,Yes,Yes,Public,No,Open WiFi,...,UPI,Yes,summer vacations,0,No,NaN,NaN,NaN,0,0.000
4,25-34,Female,Rajasthan,Mobile,WhatsApp,Yes,Yes,Private,No,Private,...,Cash,Yes,Holi,0,Yes,LinkedIn,ecommerce.com,Good,13,2.449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,18-24,Female,Tamil Nadu,Mobile,WhatsApp,Yes,No,Public,Yes,Private,...,Debit Card,Yes,Holi,0,Yes,Instagram,ecommerce.com,Good,13,2.449
4996,18-24,Male,Bihar,Tablet,Facebook,No,No,Public,Yes,Private,...,Digital Wallet,Yes,Diwali,1,No,NaN,NaN,NaN,0,0.000
4997,25-34,Male,West Bengal,Mobile,Facebook,Yes,Yes,Private,No,Hotspot,...,Cash,No,Diwali,0,No,NaN,NaN,NaN,0,0.000
4998,18-24,Female,Tamil Nadu,Desktop,LinkedIn,Yes,No,Private,No,Private,...,Credit Card,Yes,winter vacation,0,Yes,WhatsApp,news-portal.org,Good,15,3.640


In [8]:
y

0       0
1       0
2       0
3       0
4       0
       ..
4995    0
4996    0
4997    0
4998    0
4999    0
Name: Click_Leads_To_Malicious_Site, Length: 5000, dtype: int64

In [9]:
num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = []
for col in X.columns:
    if col not in num_cols:
         cat_cols.append(col)
# cat_cols = [col for col in X.columns if col not in num_cols]
cat_cols


['Age_Group',
 'Gender',
 'Location',
 'Device_Type',
 'Social_Media_Platform',
 'Two_Factor_Authentication',
 'Saved_Password',
 'Account_Privacy',
 'Frequent_New_Device_Logging',
 'Connection_Type',
 'Time_of_Day',
 'Payment_Mode',
 'Cross_Channel_Usage',
 'Season',
 'Clicked_Ad',
 'Ad_Platform',
 'Ad_Target_Domain',
 'Domain_Reputation']

In [11]:
X[num_cols] = X[num_cols].fillna(X[num_cols].median())  
X[cat_cols] = X[cat_cols].fillna(X[cat_cols].mode().iloc[0]) 

In [12]:
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

In [13]:
encoder = LabelEncoder()

for col in cat_cols:
    X[col] = encoder.fit_transform(X[col])

In [14]:
X

,Age_Group,Gender,Location,Device_Type,Social_Media_Platform,Two_Factor_Authentication,Saved_Password,Account_Privacy,Frequent_New_Device_Logging,Connection_Type,...,Payment_Mode,Cross_Channel_Usage,Season,Suspicious_Activity,Clicked_Ad,Ad_Platform,Ad_Target_Domain,Domain_Reputation,Domain_Length,Domain_Entropy
0,2,0,7,1,3,1,0,0,0,1,...,3,1,1,1.559394,1,1,0,0,1.056673,1.191224
1,0,0,17,1,0,1,1,0,0,2,...,3,0,0,1.559394,0,1,2,0,-0.907793,-0.905157
2,2,2,4,1,1,1,1,1,0,2,...,1,0,2,-0.641275,0,1,2,0,-0.907793,-0.905157
3,0,0,17,2,4,1,1,1,0,1,...,4,1,3,-0.641275,0,1,2,0,-0.907793,-0.905157
4,1,0,14,1,4,1,1,0,0,2,...,0,1,1,-0.641275,1,2,2,0,0.794745,0.558783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,0,15,1,4,1,0,1,1,2,...,2,1,1,-0.641275,1,1,2,0,0.794745,0.558783
4996,0,1,2,2,0,0,0,1,1,2,...,3,1,0,1.559394,0,1,2,0,-0.907793,-0.905157
4997,1,1,18,1,0,1,1,0,0,0,...,0,0,0,-0.641275,0,1,2,0,-0.907793,-0.905157
4998,0,0,15,0,2,1,0,0,0,2,...,1,1,4,-0.641275,1,4,4,0,1.056673,1.270728


In [15]:
y

0       0
1       0
2       0
3       0
4       0
       ..
4995    0
4996    0
4997    0
4998    0
4999    0
Name: Click_Leads_To_Malicious_Site, Length: 5000, dtype: int64

In [17]:
# 1️⃣ Split into train, validation, and test sets
# First split: train (60%) + temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

# Second split: validation (20%) + test (20%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Train size: {len(X_train)}, Validation size: {len(X_val)}, Test size: {len(X_test)}")

# 2️⃣ Logistic Regression
logistic_model = LogisticRegression(max_iter=1000, random_state=42)
logistic_model.fit(X_train, y_train)

# Evaluate on Validation set
y_val_pred = logistic_model.predict(X_val)
y_val_proba = logistic_model.predict_proba(X_val)[:, 1]

print("\n📊 Logistic Regression (Validation Set)")
print(f"Accuracy : {accuracy_score(y_val, y_val_pred):.3f}")
print(f"Precision: {precision_score(y_val, y_val_pred):.3f}")
print(f"Recall   : {recall_score(y_val, y_val_pred):.3f}")
print(f"F1 Score : {f1_score(y_val, y_val_pred):.3f}")
print(f"ROC AUC  : {roc_auc_score(y_val, y_val_proba):.3f}")

# Final test evaluation (after tuning if needed)
y_test_pred = logistic_model.predict(X_test)
y_test_proba = logistic_model.predict_proba(X_test)[:, 1]

print("\n📈 Logistic Regression (Test Set)")
print(f"Accuracy : {accuracy_score(y_test, y_test_pred):.3f}")
print(f"Precision: {precision_score(y_test, y_test_pred):.3f}")
print(f"Recall   : {recall_score(y_test, y_test_pred):.3f}")
print(f"F1 Score : {f1_score(y_test, y_test_pred):.3f}")
print(f"ROC AUC  : {roc_auc_score(y_test, y_test_proba):.3f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))

# 3️⃣ Random Forest
Random_Forest_model = RandomForestClassifier(n_estimators=200, random_state=42)
Random_Forest_model.fit(X_train, y_train)

# Validation evaluation
y_val_pred = Random_Forest_model.predict(X_val)
y_val_proba = Random_Forest_model.predict_proba(X_val)[:, 1]

print("\n🌲 Random Forest (Validation Set)")
print(f"Accuracy : {accuracy_score(y_val, y_val_pred):.3f}")
print(f"Precision: {precision_score(y_val, y_val_pred):.3f}")
print(f"Recall   : {recall_score(y_val, y_val_pred):.3f}")
print(f"F1 Score : {f1_score(y_val, y_val_pred):.3f}")
print(f"ROC AUC  : {roc_auc_score(y_val, y_val_proba):.3f}")

# Final test evaluation
y_test_pred = Random_Forest_model.predict(X_test)
y_test_proba = Random_Forest_model.predict_proba(X_test)[:, 1]

print("\n✅ Random Forest (Test Set)")
print(f"Accuracy : {accuracy_score(y_test, y_test_pred):.3f}")
print(f"Precision: {precision_score(y_test, y_test_pred):.3f}")
print(f"Recall   : {recall_score(y_test, y_test_pred):.3f}")
print(f"F1 Score : {f1_score(y_test, y_test_pred):.3f}")
print(f"ROC AUC  : {roc_auc_score(y_test, y_test_proba):.3f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))


Train size: 3000, Validation size: 1000, Test size: 1000

📊 Logistic Regression (Validation Set)
Accuracy : 0.903
Precision: 0.764
Recall   : 0.775
F1 Score : 0.770
ROC AUC  : 0.951

📈 Logistic Regression (Test Set)
Accuracy : 0.892
Precision: 0.729
Recall   : 0.764
F1 Score : 0.746
ROC AUC  : 0.943
Confusion Matrix:
 [[733  59]
 [ 49 159]]

🌲 Random Forest (Validation Set)
Accuracy : 0.943
Precision: 0.809
Recall   : 0.952
F1 Score : 0.875
ROC AUC  : 0.975

✅ Random Forest (Test Set)
Accuracy : 0.932
Precision: 0.778
Recall   : 0.942
F1 Score : 0.852
ROC AUC  : 0.970
Confusion Matrix:
 [[736  56]
 [ 12 196]]


In [19]:
# "XGBoost": XGBClassifier(n_estimators=150, use_label_encoder=False, eval_metric="logloss", scale_pos_weight=(sum(y_train==0)/sum(y_train==1)), random_state=42)
XGBoost_model = XGBClassifier(n_estimators=150, eval_metric="logloss", random_state=42)
XGBoost_model.fit(X_train, y_train)
y_pred = XGBoost_model.predict(X_test)
y_proba = XGBoost_model.predict_proba(X_test)[:, 1]

In [20]:
# Validation evaluation
y_val_pred = XGBoost_model.predict(X_val)
y_val_proba = XGBoost_model.predict_proba(X_val)[:, 1]

print("\n🌲 XGBoost (Validation Set)")
print(f"Accuracy : {accuracy_score(y_val, y_val_pred):.3f}")
print(f"Precision: {precision_score(y_val, y_val_pred):.3f}")
print(f"Recall   : {recall_score(y_val, y_val_pred):.3f}")
print(f"F1 Score : {f1_score(y_val, y_val_pred):.3f}")
print(f"ROC AUC  : {roc_auc_score(y_val, y_val_proba):.3f}")


🌲 XGBoost (Validation Set)
Accuracy : 0.924
Precision: 0.818
Recall   : 0.818
F1 Score : 0.818
ROC AUC  : 0.969


In [21]:
print(f"Accuracy : {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall   : {recall_score(y_test, y_pred)}")
print(f"F1 Score : {f1_score(y_test, y_pred)}")
print(f"ROC AUC  : {roc_auc_score(y_test, y_proba)}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy : 0.919
Precision: 0.777292576419214
Recall   : 0.8557692307692307
F1 Score : 0.8146453089244852
ROC AUC  : 0.9662126068376068
Confusion Matrix:
 [[741  51]
 [ 30 178]]
